## Some heading to talk about project

In [6]:
import numpy as np
import pptk
import random

# Load the point cloud data
target_data = []
filename = 'ENVISAT.fbx.xyz'
with open(filename,'r') as f:
    for line in f.readlines():
        line_data = line.split()
        x, y, z = line_data
        target_data.append([float(x), float(y), float(z)])
target_data = np.asarray(target_data)

In [5]:
# Visualize the target model
v = pptk.viewer(target_data)
v.set(lookat=(0,0,-5))

### Generating Point Cloud Data


In [34]:
# Transformation matrices
def transform(phi, theta, gamma):
    ''' Creates the transform matrix R
    ARGS:
        phi (rad):      rotation about x-axis
        theta (rad):    rotation about y-axis
        gamma(rad):     rotation about z-axis
    RETURN:
        R:              3x3 rotation matrix
    '''
    R_x = np.array([
        [1,  0,            0          ],
        [0,  np.cos(phi),  np.sin(phi)],
        [0, -np.sin(phi),  np.cos(phi)]
    ])
    R_y = np.array([
        [np.cos(theta), 0, -np.sin(theta)],
        [0,             1,  0            ],
        [np.sin(theta), 0,  np.cos(theta)]
    ])
    R_z = np.array([
        [ np.cos(gamma), np.sin(gamma), 0],
        [-np.sin(gamma), np.cos(gamma), 0],
        [0,              0,             1]
    ])

    R = np.cross(np.cross(R_y, R_x), R_z)
    return R

def _calculate_error(xyz, err):
    # model error with gaussian distribution, zero mean, std dev of err
    gauss_err = random.gauss(0, err)
    if gauss_err > err or gauss_err < -err:
        gauss_err = err

    dist_squared = xyz[0]**2 + xyz[1]**2 + xyz[2]**2 + err
    axis = random.choice([0, 1, 2]) # axis on which to apply the error

    new_dist = 0
    adjust = err
    thresh = err/2
    for _ in range(30):
        if axis == 0:
            new_dist = (xyz[0] + adjust)**2 + xyz[1]**2 + xyz[2]**2
        elif axis == 1:
            new_dist = xyz[0]**2 + (xyz[1] + adjust)**2 + xyz[2]**2
        if axis == 2:
            new_dist = xyz[0]**2 + xyz[1]**2 + (xyz[2] + adjust)**2 

        if new_dist < dist_squared + thresh and new_dist > dist_squared - thresh:
            break
        elif new_dist > dist_squared + thresh:
            adjust -= thresh
        elif new_dist < dist_squared - thresh:
            adjust += thresh

    # Update the axis that has the error
    for i in range(3):
        if i == axis:
            xyz[i] += adjust
    return xyz

def simulated_position(xyz, T_0, R, alpha_h, alpha_v, d_err):
    ''' Given target model computes the simulated sensor data
    ARGS:
        xyz (Nx3):      point cloud data of the target model
        T_0 (3x1):      relative position
        R:              3x3 rotation matrix
        alpha_h (rad):  horizontal observable range
        alpha_v (rad):  vertical observable range
        d_err (m):     max sensor distance error
    '''
    T = np.zeros([3,1])
    new_xyz = []
    for entry in xyz:
        # Each entry contains x,y,z
        entry_T = entry.reshape(3,1) # transpose data into column vector
        T = T_0 + entry_T
        sim = np.dot(R, entry_T) + T

        # Check if the new point is within the sensor range
        H_range = sim[0] * np.tan(alpha_h / 2) # y
        V_range = sim[0] * np.tan(alpha_v / 2) # z
        if sim[1] < H_range and sim[1] > -H_range and \
            sim[2] < V_range and sim[2] > -V_range:
            # Add distance error
            sim = _calculate_error(sim, d_err)
            new_xyz.append([sim[0], sim[1], sim[2]])
    return new_xyz


In [35]:
alpha_h = np.deg2rad(43.6)  # obtained from sensor spec
alpha_v = np.deg2rad(34.6)  # obtained from sensor spec
dist_error = 0.01           # 1 cm chosen as per paper

# Reproducing Experiment 1
#   (x,y,z) = (10,0,0) meters
#   (phi, theta, gamma) = (-180,0,0) degrees
T_0 = np.array([ [10], [0], [0] ])
R = transform(np.deg2rad(-180), 0, 0)
new_point_cloud = simulated_position(target_data, T_0, R, alpha_h, alpha_v, dist_error)

v = pptk.viewer(new_point_cloud)
v.set(lookat=(0,0,-5))

In [9]:
# Reproducing Experiment 2
#   (x,y,z) = (2,0,0) meters
#   (phi, theta, gamma) = (-180,0,0) degrees
T_0 = np.array([ [2], [0], [0] ])
R = transform(np.deg2rad(-180), 0, 0)
new_point_cloud = simulated_position(target_data, T_0, R, alpha_h, alpha_v, dist_error)

v = pptk.viewer(new_point_cloud)
v.set(lookat=(0,0,-5))